# Text Generation

In [2]:
import numpy as np 
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

## Step 1 : Get the Corpus

In [3]:
path = tf.keras.utils.get_file('irish-lyrics-eof.txt', 
                               'https://storage.googleapis.com/laurencemoroney-blog.appspot.com/irish-lyrics-eof.txt')
print (path)

data = open(path).read()

corpus = data.lower().split("\n")

C:\Users\Amir\.keras\datasets\irish-lyrics-eof.txt


## Step 2 : Tokenize the Corpus

In [4]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts(corpus)

total_words = len(tokenizer.word_index) + 1

#print(tokenizer.word_index)
print(total_words)


2690


## Step 3 : Create  n-Gram

In [5]:
input_sequences = []

for line in corpus:
    #print(line)
    token_list = tokenizer.texts_to_sequences([line])[0]
    #print(token_list)
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        #print(n_gram_sequence)
        input_sequences.append(n_gram_sequence)

## Step 4 : Pad sequences 

In [6]:
# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

## Step 5 : X and y Values

In [7]:
# label one hot encoded
x_train  = input_sequences[:,:-1]
y_label = input_sequences[:,-1]

y_train = tf.keras.utils.to_categorical(y_label, num_classes=total_words)

### Explore data

In [8]:
print(tokenizer.word_index['in'])
print(tokenizer.word_index['the'])
print(tokenizer.word_index['town'])
print(tokenizer.word_index['of'])
print(tokenizer.word_index['athy'])
print(tokenizer.word_index['one'])
print(tokenizer.word_index['jeremy'])
print(tokenizer.word_index['lanigan'])

8
1
71
6
713
39
1790
1791


In [9]:
print(x_train[6])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 2]


In [10]:
print(y_train[6])

[0. 0. 0. ... 0. 0. 0.]


In [11]:
print(x_train[5])
print(y_train[5])

[   0    0    0    0    0    0    0    0    0   51   12   96 1217   48
    2]
[0. 0. 0. ... 0. 0. 0.]


In [12]:
#print(tokenizer.word_index)

## Step 6 : Create Model

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
#earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
history = model.fit(x_train, y_train, epochs=100, verbose=1)
#print model.summary()
print(model)


Epoch 1/100
377/377 [==============================] - 14s 37ms/step - loss: 6.6774 - accuracy: 0.0705
Epoch 2/100
377/377 [==============================] - 14s 38ms/step - loss: 5.8152 - accuracy: 0.1068
Epoch 3/100
377/377 [==============================] - 13s 35ms/step - loss: 4.9973 - accuracy: 0.1569
Epoch 4/100
377/377 [==============================] - 13s 35ms/step - loss: 4.1011 - accuracy: 0.2226
Epoch 5/100
377/377 [==============================] - 14s 37ms/step - loss: 3.2412 - accuracy: 0.3200
Epoch 6/100
377/377 [==============================] - 16s 42ms/step - loss: 2.5379 - accuracy: 0.4309
Epoch 7/100
377/377 [==============================] - 16s 43ms/step - loss: 2.0100 - accuracy: 0.5319
Epoch 8/100
377/377 [==============================] - 16s 41ms/step - loss: 1.6489 - accuracy: 0.6053
Epoch 9/100
377/377 [==============================] - 14s 38ms/step - loss: 1.3757 - accuracy: 0.6688
Epoch 10/100
377/377 [==============================] - 14s 37ms/step - l

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.show()

In [ ]:
plot_graphs(history, 'accuracy')


## Step 7 : Generate Text

In [ ]:
seed_text = "I've got a bad feeling about this"
next_words = 100
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = np.argmax(model.predict(token_list), axis=-1)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)